<a href="https://colab.research.google.com/github/Denev6/CapStone/blob/main/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchsummary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# CNN 살펴보기

In [ ]:
import os
import gc
import pickle
import random
from google.colab import drive
# import warnings

import numpy as np 
import pandas as pd
from tqdm.auto import tqdm, trange
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
)

In [ ]:
drive.mount("/content/drive")
# warnings.simplefilter("ignore")

def clear():
    gc.collect()
    torch.cuda.empty_cache()

def join_path(*args):
    return os.path.join("/content/drive/MyDrive", *args)

train_csv = join_path("Capstone", "data", "train.pk")
dev_csv = join_path("Capstone", "data", "dev.pk")
test_csv = join_path("Capstone", "data", "test.pk")

train_data = pickle.load(open(train_csv, 'rb'))
dev_data = pickle.load(open(dev_csv, 'rb'))
test_data = pickle.load(open(test_csv, 'rb'))

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 4
EPOCHS = 5
LEARNING_RATE = 1e-4
MODEL_PATH = join_path("tdcn.pth")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Dataset

In [ ]:
class CustomDataset(Dataset):
    """데이터 처리"""
    def __init__(self, data, up_count=500, data_size=5000):
        self.data = {}
        for k, v in data.items():
            tmp = {
                'pose' : torch.FloatTensor(v['pose'].astype('float').values),
                'features' : torch.FloatTensor(v['features'].astype('float').values),
                'y' : torch.IntTensor([v['y']])
            }
            self.data[k] = tmp
        self.up_count = up_count
        self.data_size = data_size
        
    def __len__(self):
        return len(self.data) * self.up_count

    def __getitem__(self, index):
        target_data = random.choice(list(self.data.values()))
        x1 = target_data['features']
        x2 = target_data['pose']
        cut = random.randrange(0, len(x1)-self.data_size)
        x1 = x1[cut: cut + self.data_size]
        x2 = x2[cut: cut + self.data_size]
        y = target_data['y']
        return (x1, x2), y

In [ ]:
# 데이터셋
training_dataset = CustomDataset(train_data)
dev_dataset = CustomDataset(dev_data, up_count=1)
test_dataset = CustomDataset(test_data, up_count=10)

train_dataloader = DataLoader(training_dataset, batch_size=BATCH_SIZE)
dev_dataloader = DataLoader(dev_dataset, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

<ipython-input-4-803cc82a778a>:9: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  'y' : torch.IntTensor([v['y']])


## CNN

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, num_label=2):
        super(BaseModel, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=2, dilation=2, padding="same"),
            nn.ELU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(8, 16, kernel_size=2, dilation=2, padding="same"),
            nn.ELU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(16, affine=True)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(1, -1),
            nn.Linear(350000 * 2, 1024),
            nn.ELU(inplace=True),
            nn.Linear(1024, 64),
            nn.ELU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(64, num_label),
        )

    def forward(self, feat_x, pose_x):
        feat_y = self.conv(feat_x)
        pose_y = self.conv(pose_x)
        out = torch.cat((feat_y, pose_y), dim=-1)
        out = self.classifier(out)
        return out

In [ ]:
model = BaseModel(num_label=2)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)

In [ ]:
model.to(DEVICE)
summary(model, [(1, 5000, 136), (1, 5000, 6)], batch_size=BATCH_SIZE)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [4, 8, 5000, 136]              40
               ELU-2          [4, 8, 5000, 136]               0
         MaxPool2d-3           [4, 8, 2500, 68]               0
            Conv2d-4          [4, 16, 2500, 68]             528
               ELU-5          [4, 16, 2500, 68]               0
         MaxPool2d-6          [4, 16, 1250, 34]               0
       BatchNorm2d-7          [4, 16, 1250, 34]              32
            Conv2d-8            [4, 8, 5000, 6]              40
               ELU-9            [4, 8, 5000, 6]               0
        MaxPool2d-10            [4, 8, 2500, 3]               0
           Conv2d-11           [4, 16, 2500, 3]             528
              ELU-12           [4, 16, 2500, 3]               0
        MaxPool2d-13           [4, 16, 1250, 1]               0
      BatchNorm2d-14           [4, 16, 

## Train

In [ ]:
def train(train_loader, test_loader, model, loss_fn, optimizer):
    model.to(DEVICE) 
    loss_fn.to(DEVICE)
    optimizer.zero_grad()
    num_batches = len(train_loader)

    epoch_progress = trange(1, EPOCHS + 1)
    # tqdm.write("\nEpoch | Train Loss | Test Loss")
    # tqdm.write("-" * 30)
    
    for epoch in epoch_progress:
        model.train()
        # train_loss = 0
        for (x1, x2), label in train_loader:
            x1 = x1.unsqueeze(1).to(DEVICE) 
            x2 = x2.unsqueeze(1).to(DEVICE) 
            label = label.squeeze(-1).to(DEVICE)
            # x1: [8, 1, 5000, 136] > [Batch, Channel, Height, Width]
            # x2: [8, 1, 5000, 6] > [Batch, Channel, Height, Width]
            # label: [8] > [Height]
            
            pred = model(x1, x2)  
            # pred: [8, 2]
            loss = loss_fn(pred, label.long())
            # train_loss += loss.item()
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # train_loss /= num_batches
    
        # eval_loss = evaluate(test_loader, model, loss_fn)
        # tqdm.write(f"{epoch:5} | {train_loss:10.5f} | {eval_loss:9.5f}")

# 학습된 모델 불러오기
    return model


def evaluate(loader, model, loss_fn, return_metrics=False):
    model.eval()
    num_batches = len(loader)
    test_loss = 0
    true_labels = list()
    pred_values = list()

    with torch.no_grad():
        for (x1, x2), label in loader:
            x1 = x1.unsqueeze(1).to(DEVICE) 
            x2 = x2.unsqueeze(1).to(DEVICE) 
            label = label.squeeze(-1).to(DEVICE)

            pred = model(x1, x2)  
            loss = loss_fn(pred, label.long())
            test_loss += loss.item()

            if return_metrics:
                true_labels += label.detach().cpu().numpy().tolist()
                pred_values += pred.argmax(-1).detach().cpu().numpy().tolist()

    if not return_metrics:
        # 학습 과정에서는 Loss 값만 확인합니다.
        test_loss /= num_batches
        return test_loss

    else:
        # 학습이 종료되고 성능 평가 지표를 확인합니다.
        accuracy = accuracy_score(true_labels, pred_values)
        f1 = f1_score(true_labels, pred_values)
        f1_macro = f1_score(true_labels, pred_values, average="macro")
        recall = recall_score(true_labels, pred_values)
        precision = precision_score(true_labels, pred_values)
        matrix = confusion_matrix(true_labels, pred_values).ravel()

        return {
            "accuracy": accuracy,
            "f1": f1,
            "f1-macro": f1_macro,
            "recall": recall,
            "precision": precision,
            "loss": test_loss,
            "matrix": matrix,
        }

In [ ]:
# 모델 학습
model = train(train_dataloader, dev_dataloader, model, loss_fn, optimizer)

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
torch.save(model.state_dict(), join_path(MODEL_PATH))

## Test

In [ ]:
clear()
metrics = evaluate(test_dataloader, model, loss_fn, return_metrics=True)

print(f"Accuracy:  {metrics['accuracy']:.3f}")
print(f"F1-score:  {metrics['f1']:.3f}")
print(f"F1-macro:  {metrics['f1-macro']:.3f}")
print(f"Recall:    {metrics['recall']:.3f}")
print(f"Precision: {metrics['precision']:.3f}")

print("-" * 30)
tn, fp, fn, tp = metrics["matrix"]
print(f"TN: {tn}")
print(f"FP: {fp}")
print(f"FN: {fn}")
print(f"TP: {tp}")

Accuracy:  0.532
F1-score:  0.091
F1-macro:  0.388
Recall:    0.048
Precision: 1.000
------------------------------
TN: 239
FP: 0
FN: 220
TP: 11


In [ ]:
def show_probs(test_data, model, max=6):
    dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

    model.eval()
    neg_max = max // 2
    pos_max = max - neg_max
    pos_count = 0
    neg_count = 0
    with torch.no_grad():
        for (x1, x2), label in dataloader:
            x1 = x1.unsqueeze(1).to(DEVICE) 
            x2 = x2.unsqueeze(1).to(DEVICE) 
            # label = label.to(DEVICE)

            if label.item() == 0 and pos_count < pos_max:
                pos_count += 1
                label = label.item()
            elif label.item() == 1 and neg_count < neg_max:
                neg_count += 1
                label = label.item()
            elif pos_count + neg_count == max:
                break
            else:
                continue

            pred = model(x1, x2)
            pred = F.softmax(pred.squeeze(0), dim=-1)
            normal, abnormal = pred
            print(f"{label}: [{normal:.3f}  {abnormal:.3f}]")

show_probs(test_data, model, 30)

1: [0.636  0.364]
0: [1.000  0.000]
0: [1.000  0.000]
1: [0.998  0.002]
1: [1.000  0.000]
0: [1.000  0.000]
0: [1.000  0.000]
0: [1.000  0.000]
0: [1.000  0.000]
0: [1.000  0.000]
1: [0.877  0.123]
0: [1.000  0.000]
1: [1.000  0.000]
1: [1.000  0.000]
0: [1.000  0.000]
1: [1.000  0.000]
1: [0.996  0.004]
0: [1.000  0.000]
0: [1.000  0.000]
1: [1.000  0.000]
0: [1.000  0.000]
0: [1.000  0.000]
0: [1.000  0.000]
0: [1.000  0.000]
1: [1.000  0.000]
1: [1.000  0.000]
1: [0.931  0.069]
1: [1.000  0.000]
1: [1.000  0.000]
1: [1.000  0.000]
